# Importando Bibliotecas

In [ ]:
import pandas as pd
import requests
import re
import time
from tqdm.notebook import tqdm
import numpy as np
from datetime import date
from datetime import timedelta
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

# Lista dos Criadores

In [ ]:
# Importando lista dos criadores com maior volume [eth] no opensea nos ultimos  7 dias 
s = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=s)
driver.implicitly_wait(1.5)
driver.get('https://www.finder.com.au/cryptocurrency/nft-collections')

In [ ]:
link_html = driver.find_element(By.XPATH,'''//*[@id="main"]/div[4]/div[2]/div[2]/div/div[2]/ul/li[2]/a''')
link_html.click()

In [ ]:
driver.find_element(By.XPATH, f'''//*[@id="pb-tabname1"]/div/table/tbody/tr[1]/td[4]''').text

In [ ]:
lista_name = []
lista_sales =[]
lista_volume =[]


for i in range(1,26):
    lista_name.append(driver.find_element(By.XPATH, f'''//*[@id="pb-tabname1"]/div/table/tbody/tr[{i}]/td[2]/div/a''').text)
    lista_sales.append(driver.find_element(By.XPATH, f'''//*[@id="pb-tabname1"]/div/table/tbody/tr[{i}]/td[3]''').text)
    lista_volume.append(re.sub(' [A-Z]+','',driver.find_element(By.XPATH, f'''//*[@id="pb-tabname1"]/div/table/tbody/tr[{i}]/td[4]''').text))

In [ ]:
## Visualização daos 25 primeiros criadores da lista 

In [ ]:
df = pd.DataFrame({'creator':lista_name , 'sales':lista_sales, 'volume [ETH]':lista_volume} )
df

# Extração dos Dados


In [ ]:
#Determinando o periodo de retirada de dados
current_date = date.today().isoformat()   
days_before = (date.today()-timedelta(days=7)).isoformat()

In [ ]:
#Criando as listas para o dataframe
lista_address_aux1 = []
lista_creators1 = []
lista_tokenid1 = []
lista_price1 = []
lista_block1 = []
lista_block_minter1 = []
lista_contract1 = []


In [ ]:
# Looping para extração dos dados
for i in lista_name:
    driver.get('https://opensea.io/')
    driver.implicitly_wait(10)
    time.sleep(2)
    buscador = driver.find_element(By.XPATH, '''//*[@id="__next"]/div/div[1]/nav/div[2]/div/div/div/input''')
    buscador.send_keys(i)
    time.sleep(2)
    primeiro_item = driver.find_element(By.XPATH,f'''//*[contains(text(),'{i}')]''')
    primeiro_item.click()
    time.sleep(3)
    primeiro_nft = driver.find_element(By.XPATH,'''//*[@id="main"]/div/div/div[3]/div/div/div/div[3]/div[3]/div[2]/div/div/div[1]/div/article/a/div[2]''')
    primeiro_nft.click()
    address = driver.current_url
    contract = re.findall('[0-9]\w+',address)
    
    
    endpoint = f'https://deep-index.moralis.io/api/v2/nft/{contract[0]}/trades'
    query = {'chain':'eth','marketplace':'opensea','from_date':f'{days_before}','to_date':f'{current_date}'}
    header = {'accept' : 'application/json', 'X-API-Key' : 'zI2h59sDw8RlpIkIierrdsqUJVD8GvU3bu6UcxwvI4NQLBoj1qazAqSf0ugb6iZa'}
    resposta = requests.request('GET', endpoint, headers=header, params=query)
    resultados = resposta.json()
    time.sleep(3)
    for items in resultados['result']:
        lista_tokenid1.append(items['token_ids'][0])
        
        lista_block1.append(items['block_number'])
        lista_address_aux1.append(contract[0])
        lista_creators1.append(i)
        transaction_hash = items['transaction_hash']
        driver.get('https://etherscan.io/')
        time.sleep(1)
        buscador1 = driver.find_element(By.XPATH,'''//*[@id="txtSearchInput"]''')
        buscador1.send_keys(transaction_hash)
        buscador1.send_keys(Keys.ENTER)
        time.sleep(1)
        try:
            preco = driver.find_element(By.XPATH,'''//*[@id="wrapperContent"]/li/div/span[4]''')
            lista_price1.append(preco.text)
            moeda = driver.find_element(By.XPATH,'''//*[@id="wrapperContent"]/li/div/a''')
        except NoSuchElementException:
            preco = driver.find_element(By.XPATH,'''//*[@id="ContentPlaceHolder1_spanValue"]/span''')
            lista_price1.append(preco.text)
            
        endpoint2 = f"https://deep-index.moralis.io/api/v2/nft/{contract[0]}/{items['token_ids'][0]}"
        query2 = {'chain':'eth', 'format':'decimal'}
        header2 = {'accept' : 'application/json', 'X-API-Key' : 'zI2h59sDw8RlpIkIierrdsqUJVD8GvU3bu6UcxwvI4NQLBoj1qazAqSf0ugb6iZa'}
        response = requests.request('GET', endpoint2, headers=header2, params=query2)
        results = response.json()
        try:
            lista_block_minter1.append(results['block_number_minted'])
        except :
            lista_block_minter1.append(np.nan)
        try:
            lista_contract1.append(results['contract_type'])
        except :
            lista_contract1.append(np.nan)

In [ ]:
#Criando DataFrame
df1 = pd.DataFrame({'Criador':lista_creators1, 'Contract Address':lista_address_aux1, 'Token Id': lista_tokenid1,'Preço[ETH]' :lista_price1,'Block Chain': lista_block1})

In [ ]:
# Tirando linhas duplicadas
df_nft = df1.drop_duplicates()
df_nft

# Exportando como CSV

In [ ]:
df_nft.to_csv(r'C:\Users\guede\projetos\df_nft2.csv', sep = ';',index = False)
